<a href="https://colab.research.google.com/github/sdr999/MangaSensei/blob/main/Scraping_Data_from_mangakakalot_com.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Scraping Data from mangakakalot.com and generating DataSet**

In [35]:
import requests
from bs4 import BeautifulSoup
import pandas as pd

manga_list = []
def scrape_manga_data():
    for i in range(1,10):
      url = "https://mangakakalot.com/manga_list?type=latest&category=all&state=all&page="+str(i)
      headers = {
          "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.3"
      }
      response = requests.get(url, headers=headers)
      soup = BeautifulSoup(response.content, "html.parser")

      results = soup.find_all("div", class_="list-truyen-item-wrap")

      for result in results:
          title_tag = result.find("h3")
          description_tag = result.find("p")
          link_tag = title_tag.find("a").get("href")
          if title_tag:
              title = title_tag.text.strip()
              description = description_tag.text.strip()

              manga = {
                  "title": title,
                  "description": description,
                  "link":link_tag
              }
              manga_list.append(manga)

    return manga_list

manga_data = scrape_manga_data()
df = pd.DataFrame(manga_data)

# Save the dataset to a CSV file
df.to_csv("manga_dataset.csv", index=False)


# **Scraping Data from mangakakalot.com and generating DataSet with genres**

In [36]:
import requests
from bs4 import BeautifulSoup
import pandas as pd

def scrape_manga_genres(url):
    headers = {
        "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.3"
    }
    response = requests.get(url, headers=headers)
    soup = BeautifulSoup(response.content, "html.parser")

    genre_tags = soup.find_all("td", class_="table-value")
    genres = [genre.text.strip() for genre in genre_tags]

    return genres

# Read the dataset from CSV file
df = pd.read_csv("manga_dataset.csv")

# Scrape genres for each manga link
manga_genres = []
for link in df["link"]:
    genres = scrape_manga_genres(link)
    manga_genres.append(genres)

# Add the manga genres to the dataset
df["genres"] = manga_genres

# Save the updated dataset to a new CSV file
df.to_csv("manga_dataset_with_genres.csv", index=False)
